## Imports

In [24]:
import pandas as pd
import numpy as np
import plotly.express as px

from os import listdir, getcwd
from os.path import join, split

from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

## Reading the data

In [2]:
PATH_TO_DATA = join(split(getcwd())[0], 'data')
print(PATH_TO_DATA)

/Users/namestnikovnik/Documents/courses/mlops_zoomcamp/data


In [3]:
listdir(PATH_TO_DATA)

['yellow_tripdata_2022-01.parquet', 'yellow_tripdata_2022-02.parquet']

In [4]:
file_nm = 'yellow_tripdata_2022-01.parquet'
file_path = join(PATH_TO_DATA, file_nm)
taxi_jan_df = pd.read_parquet(file_path)
print(taxi_jan_df.shape)
taxi_jan_df.head()

(2463931, 19)


,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,airport_fee
0,1,2022-01-01 00:35:40,2022-01-01 00:53:29,2.0,3.80,1.0,N,142,236,1,14.5,3.0,0.5,3.65,0.0,0.3,21.95,2.5,0.0
1,1,2022-01-01 00:33:43,2022-01-01 00:42:07,1.0,2.10,1.0,N,236,42,1,8.0,0.5,0.5,4.00,0.0,0.3,13.30,0.0,0.0
2,2,2022-01-01 00:53:21,2022-01-01 01:02:19,1.0,0.97,1.0,N,166,166,1,7.5,0.5,0.5,1.76,0.0,0.3,10.56,0.0,0.0
3,2,2022-01-01 00:25:21,2022-01-01 00:35:23,1.0,1.09,1.0,N,114,68,2,8.0,0.5,0.5,0.00,0.0,0.3,11.80,2.5,0.0
4,2,2022-01-01 00:36:48,2022-01-01 01:14:20,1.0,4.30,1.0,N,68,163,1,23.5,0.5,0.5,3.00,0.0,0.3,30.30,2.5,0.0


In [5]:
file_nm = 'yellow_tripdata_2022-02.parquet'
file_path = join(PATH_TO_DATA, file_nm)
taxi_feb_df = pd.read_parquet(file_path)
print(taxi_feb_df.shape)
taxi_feb_df.head()

(2979431, 19)


,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,airport_fee
0,1,2022-02-01 00:06:58,2022-02-01 00:19:24,1.0,5.40,1.0,N,138,252,1,17.0,1.75,0.5,3.90,0.00,0.3,23.45,0.0,1.25
1,1,2022-02-01 00:38:22,2022-02-01 00:55:55,1.0,6.40,1.0,N,138,41,2,21.0,1.75,0.5,0.00,6.55,0.3,30.10,0.0,1.25
2,1,2022-02-01 00:03:20,2022-02-01 00:26:59,1.0,12.50,1.0,N,138,200,2,35.5,1.75,0.5,0.00,6.55,0.3,44.60,0.0,1.25
3,2,2022-02-01 00:08:00,2022-02-01 00:28:05,1.0,9.88,1.0,N,239,200,2,28.0,0.50,0.5,0.00,3.00,0.3,34.80,2.5,0.00
4,2,2022-02-01 00:06:48,2022-02-01 00:33:07,1.0,12.16,1.0,N,138,125,1,35.5,0.50,0.5,8.11,0.00,0.3,48.66,2.5,1.25


## Solutions

### Q1
> Read the data for January. How many columns are there?

**Answer**: 19

### Q2
> Now let's compute the duration variable. It should contain the duration of a ride in minutes. What's the standard deviation of the trips duration in January?

**Answer:** 46.45

In [11]:
taxi_jan_df = taxi_jan_df.assign(duration = (taxi_jan_df['tpep_dropoff_datetime'] - taxi_jan_df['tpep_pickup_datetime']).dt.total_seconds() / 60)

In [12]:
taxi_jan_df.describe(percentiles=[.01, .05, .1, .25, .5, .75, .9, .95, .99]).T

,count,mean,min,1%,5%,10%,25%,50%,75%,90%,95%,99%,max,std
VendorID,2463931.0,1.707819,1.0,1.0,1.0,1.0,1.0,2.0,2.0,2.0,2.0,2.0,6.0,0.502137
tpep_pickup_datetime,2463931,2022-01-17 01:19:51.689726208,2008-12-31 22:23:09,2022-01-01 11:23:47,2022-01-03 01:11:04.500000,2022-01-04 17:30:25,2022-01-09 15:37:41,2022-01-17 12:11:45,2022-01-24 13:49:37.500000,2022-01-28 14:27:18,2022-01-30 14:53:19,2022-01-31 18:11:00,2022-05-18 20:41:57,NaN
tpep_dropoff_datetime,2463931,2022-01-17 01:34:04.421900544,2008-12-31 23:06:56,2022-01-01 11:37:00.500000,2022-01-03 01:36:30,2022-01-04 17:44:02,2022-01-09 15:50:50.500000,2022-01-17 12:23:49,2022-01-24 14:02:51,2022-01-28 14:44:17,2022-01-30 15:07:09.500000,2022-01-31 18:25:39.700000,2022-05-18 20:47:45,NaN
passenger_count,2392428.0,1.389453,0.0,0.0,1.0,1.0,1.0,1.0,1.0,2.0,3.0,6.0,9.0,0.982969
trip_distance,2463931.0,5.372751,0.0,0.0,0.5,0.68,1.04,1.74,3.13,7.17,11.8,19.7,306159.28,547.871404
RatecodeID,2392428.0,1.415507,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,4.0,99.0,5.917573
PULocationID,2463931.0,166.076809,1.0,24.0,48.0,70.0,132.0,162.0,234.0,239.0,262.0,264.0,265.0,65.468057
DOLocationID,2463931.0,163.580716,1.0,10.0,43.0,50.0,113.0,162.0,236.0,244.0,262.0,264.0,265.0,70.790159
payment_type,2463931.0,1.194449,0.0,0.0,1.0,1.0,1.0,1.0,1.0,2.0,2.0,2.0,5.0,0.500178
fare_amount,2463931.0,12.946484,-480.0,2.5,4.5,5.0,6.5,9.0,14.0,25.0,39.5,52.5,401092.32,255.814887


### Q3
> Next, we need to check the distribution of the duration variable. There are some outliers. Let's remove them and keep only the records where the duration was between 1 and 60 minutes (inclusive).

What fraction of the records left after you dropped the outliers?

**Answer:** 98%

In [16]:
mask_keep = (taxi_jan_df['duration'] >= 1) & (taxi_jan_df['duration'] <= 60)
print(f'# records left: {sum(mask_keep)}')
print(f'% records left: {sum(mask_keep)/taxi_jan_df.shape[0]:.2f}')

taxi_jan_df = taxi_jan_df.loc[mask_keep].copy()

# records left: 2421440
% records left: 0.98


### Q4
> Let's apply one-hot encoding to the pickup and dropoff location IDs. We'll use only these two features for our model.

>Turn the dataframe into a list of dictionaries
>* Fit a dictionary vectorizer
>* Get a feature matrix from it
>* What's the dimensionality of this matrix (number of columns)?

**Answer:** 515

In [16]:
mask_keep = (taxi_jan_df['duration'] >= 1) & (taxi_jan_df['duration'] <= 60)
print(f'# records left: {sum(mask_keep)}')
print(f'% records left: {sum(mask_keep)/taxi_jan_df.shape[0]:.2f}')

taxi_jan_df = taxi_jan_df.loc[mask_keep].copy()

# records left: 2421440
% records left: 0.98


In [18]:
taxi_jan_df = taxi_jan_df.assign(PULocationID = taxi_jan_df['PULocationID'].astype(str),
                                 DOLocationID = taxi_jan_df['DOLocationID'].astype(str))


In [22]:
features = ['PULocationID', 'DOLocationID']
dv = DictVectorizer()

X_train = dv.fit_transform(taxi_jan_df[features].to_dict(orient='records'))

In [23]:
X_train.shape

(2421440, 515)

### Q5
> Now let's use the feature matrix from the previous step to train a model.

>* Train a plain linear regression model with default parameters
>* Calculate the RMSE of the model on the training data

What's the RMSE on train?

**Answer:** 6.99

In [26]:
lr = LinearRegression()
lr.fit(X=X_train, y=taxi_jan_df['duration'].values)

LinearRegression()

In [29]:
rmse_train = mean_squared_error(y_true=taxi_jan_df['duration'].values,
                                y_pred = lr.predict(X_train),
                                squared=False)
print(f'RMSE on train: {rmse_train:.2f}')

RMSE on train: 6.99


### Q6
>Now let's apply this model to the validation dataset (February 2022).

What's the RMSE on validation?

**Answer:** 7.79

In [37]:
cols_to_keep = features + ['tpep_pickup_datetime', 'tpep_dropoff_datetime']
X_test = (taxi_feb_df[cols_to_keep]
             .assign(duration = (taxi_feb_df['tpep_dropoff_datetime'] - taxi_feb_df['tpep_pickup_datetime']).dt.total_seconds() / 60,
                     PULocationID = taxi_feb_df['PULocationID'].astype(str),
                     DOLocationID = taxi_feb_df['DOLocationID'].astype(str))
             .loc[lambda df: (df['duration'] >= 1) & (df['duration'] <= 60)]
             .drop(columns=['tpep_pickup_datetime', 'tpep_dropoff_datetime'])
         )
print(X_test.shape)
y_test = X_test['duration']
X_test = X_test.drop(columns=['duration'])
print(X_test.info())
X_test.head()


(2918187, 3)
<class 'pandas.core.frame.DataFrame'>
Index: 2918187 entries, 0 to 2979430
Data columns (total 2 columns):
 #   Column        Dtype 
---  ------        ----- 
 0   PULocationID  object
 1   DOLocationID  object
dtypes: object(2)
memory usage: 66.8+ MB
None


,PULocationID,DOLocationID
0,138,252
1,138,41
2,138,200
3,239,200
4,138,125


In [38]:
X_test = dv.transform(X_test.to_dict(orient='records'))
X_test.shape

(2918187, 515)

In [39]:
rmse_test = mean_squared_error(y_true=y_test,
                               y_pred = lr.predict(X_test),
                               squared=False)
print(f'RMSE on train: {rmse_test:.2f}')

RMSE on train: 7.79
